<a href="https://colab.research.google.com/github/SatishGollamudi/NLP/blob/main/Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [5]:
df.shape


(20800, 5)

In [6]:
df=df.dropna()

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
## get the independent features
X=df.drop('label',axis=1)

In [9]:
y=df['label']

In [10]:
##Check whether the data is balanced
y.value_counts()

,count
label,
0,10361
1,7924


In [11]:
import tensorflow as tf


In [12]:
tf.__version__

'2.8.2'

In [46]:
from tensorflow import keras

In [47]:
from keras import layers

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [14]:
### Vocabulary size
voc_size=5000

In [15]:
messages=X.copy()

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
messages.reset_index(inplace=True)

In [19]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4220, 4837, 228, 4885, 3272, 2373, 4686, 4921, 4863, 764],
 [4996, 4596, 23, 2658, 2128, 3499, 769],
 [1380, 1751, 1851, 3472],
 [3901, 2600, 2936, 4010, 1804, 4861],
 [4031, 2128, 1283, 1348, 3394, 3922, 2128, 3301, 1085, 3552],
 [3382,
  3435,
  4489,
  2933,
  2418,
  80,
  2852,
  3478,
  888,
  4607,
  3358,
  2380,
  4322,
  3115,
  769],
 [2568, 1810, 1720, 2256, 1908, 2623, 916, 4285, 4744, 4385, 2542],
 [2526, 3026, 1846, 3253, 4609, 2909, 80, 1243, 4744, 4385, 2542],
 [3966, 1287, 2362, 2976, 2394, 212, 4931, 599, 80, 4957],
 [180, 235, 2815, 312, 2457, 4433, 570, 4928],
 [1556, 1973, 1616, 887, 2746, 2639, 118, 2505, 4163, 3708, 4796],
 [4010, 3363, 3272, 212, 80, 4609],
 [519, 1555, 1079, 2345, 3644, 4086, 4664, 275, 510],
 [1941, 1441, 85, 4689, 1256, 4252, 3409, 4744, 4385, 2542],
 [215, 405, 1529, 3084, 4228, 4744, 4385, 2542],
 [900, 214, 4675, 1210, 4079, 2305, 543, 2028, 4111, 3813],
 [3167, 393, 4596],
 [2069, 6, 3629, 2950, 80, 306, 1310, 769],
 [4529, 2646, 23, 1

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4921, 4863,  764],
       [   0,    0,    0, ..., 2128, 3499,  769],
       [   0,    0,    0, ..., 1751, 1851, 3472],
       ...,
       [   0,    0,    0, ..., 4744, 4385, 2542],
       [   0,    0,    0, ..., 2465, 2659, 1255],
       [   0,    0,    0, ..., 1509, 4033, 3465]], dtype=int32)

In [23]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
### Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 25s 47ms/step - loss: 0.2649 - accuracy: 0.8814 - val_loss: 0.2006 - val_accuracy: 0.9203
Epoch 2/10
383/383 [==============================] - 17s 44ms/step - loss: 0.1384 - accuracy: 0.9471 - val_loss: 0.2087 - val_accuracy: 0.9132
Epoch 3/10
383/383 [==============================] - 17s 44ms/step - loss: 0.0954 - accuracy: 0.9657 - val_loss: 0.2531 - val_accuracy: 0.9175
Epoch 4/10
383/383 [==============================] - 17s 44ms/step - loss: 0.0697 - accuracy: 0.9759 - val_loss: 0.2445 - val_accuracy: 0.9082
Epoch 5/10
383/383 [==============================] - 17s 45ms/step - loss: 0.0440 - accuracy: 0.9847 - val_loss: 0.3271 - val_accuracy: 0.9051
Epoch 6/10
383/383 [==============================] - 18s 48ms/step - loss: 0.0278 - accuracy: 0.9912 - val_loss: 0.4727 - val_accuracy: 0.9112
Epoch 7/10
383/383 [==============================] - 17s 44ms/step - loss: 0.0221 - accuracy: 0.9937 - val_loss: 0.4123 - val_accuracy:

In [27]:

y_pred=model.predict(X_test)

In [31]:
#numpy
import numpy as np
y_pred=np.where(y_pred>=0.5,1,0)

In [33]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3163,  256],
       [ 349, 2267]])

In [34]:
print(accuracy_score(y_test,y_pred))

0.899751449875725
